# Interactive collaborative web server

### 1. Create a new Seamless project
(If Seamless has been installed locally, you would use `seamless-new-project demo-webserver`)

In [1]:
import os
if not os.path.exists("web/"):
    os.system("python3 ...")

### 2. Set up a simple Seamless context

In [2]:
def add(a, b):
    return a + b

ctx.a = 10          # ctx.a => Seamless cell
ctx.b = 20          # ctx.b => Seamless cell
ctx.add = add       # ctx.add => Seamless transformer
ctx.add.a = ctx.a
ctx.add.b = ctx.b
ctx.c = ctx.add     # ctx.c => Seamless cell
await ctx.computation()
ctx.c.value

<Silk: 30 >

In [1]:
from seamless.highlevel import Context
ctx = Context()

### 2. Set up a simple Seamless context

In [2]:
def add(a, b):
    return a + b

ctx.a = 10          # ctx.a => Seamless cell
ctx.b = 20          # ctx.b => Seamless cell
ctx.add = add       # ctx.add => Seamless transformer
ctx.add.a = ctx.a
ctx.add.b = ctx.b
ctx.c = ctx.add     # ctx.c => Seamless cell
await ctx.computation()
ctx.c.value

<Silk: 30 >